## PERIN

Preprocess the data from the stancer pipeline and bring it into canonical form so it can be used by the sentiment graph parser.

In [42]:
# IMPORTS
import pandas as pd
import json
import os

In [43]:
# makedirs if not exist
os.makedirs("../../etl/data/processed/Perin_Preprocessing", exist_ok=False)

FileExistsError: [Errno 17] File exists: '../../etl/data/processed/Perin_Preprocessing'

In [44]:
TRAIN_INPUT_PATH = "../../etl/data/intermediate/TrainTestSplit/01_train.csv"
TEST_INPUT_PATH = "../../etl/data/intermediate/TrainTestSplit/01_test.csv"
VAL_INPUT_PATH = "../../etl/data/intermediate/TrainTestSplit/01_val.csv"

TRAIN_OUTPUT_PATH = "../../etl/data/processed/Perin_Preprocessing/01_train.json"
TEST_OUTPUT_PATH = "../../etl/data/processed/Perin_Preprocessing/01_test.json"
VAL_OUTPUT_PATH = "../../etl/data/processed/Perin_Preprocessing/01_val.json"

# Full phrase or Head mode
FULL_PHRASE = True

In [45]:
# read in each dataframe
input_paths = [TRAIN_INPUT_PATH, TEST_INPUT_PATH, VAL_INPUT_PATH]
dataframes = []

for ip in input_paths:
    df = pd.read_csv(ip, index_col=0)
    dataframes.append(df)


In [46]:
# inspect a df
dataframes[1]

,verb_form,verb_form_start,verb_form_end,verb_lemma,arg1,arg1_start,arg1_end,arg1_pos,arg1_head,arg1_head_start,...,arg2,arg2_start,arg2_end,arg2_pos,arg2_head,arg2_head_start,arg2_head_end,rel_type,pred_serial,full_sentence_text
doc_id,,,,,,,,,,,,,,,,,,,,,
b8a2b49dabdf4885588ac0970cf71e721e9477a7ce0c5401b01d69bf0cbff36a,verlassen,34,43,verlassen,Gut-Behrami,0,11,N,Gut-Behrami,0,...,ihre Komfortzone,17,33,N,Komfortzone,22,33,con,"Predicate(type='con', args=(Head(sentence=5, t...",Gut-Behrami muss ihre Komfortzone verlassen .
2366449839cea94e58036c461763ee4b613f08b0541d578756bab86d4f12685a,trennen,38,45,trennen,zunächst allgemeine technische Fähigkeiten,88,130,N,Fähigkeiten,119,...,Weizen,28,34,N,Weizen,28,34,con,"Predicate(type='con', args=(Head(sentence=22, ...",Um schon früh die Spreu vom Weizen zu trennen ...
cdf5a4224e30ff92ea1dc0762ed2c0b36fcac62d027a5fda9db9eecb1f6a57b4,verzichtet,13,23,verzichten,Federer,5,12,N,Federer,5,...,die Teilnahme,28,41,N,Teilnahme,32,41,con,"Predicate(type='con', args=(Head(sentence=5, t...",Denn Federer verzichtet auf die Teilnahme .
9ed3c86676abb590a342c5bb1cba3b00fa56545eaf17b9c3d39d15fb03a37e87,bestraft,30,38,bestrafen,Der Staat,20,29,N,Staat,24,...,die Armen,39,48,N,Armen,43,48,con,"Predicate(type='con', args=(Head(sentence=29, ...","MARLO BÜRGE , AARAU Der Staat bestraft die Arm..."
3f794f334f2d3ddd2b3647d5fca75e742b5f6ef31a25d013d48adab59db9de27,verbessere,19,29,verbessern,Sein Alkoholkonsum,0,18,N,Alkoholkonsum,5,...,seine Prognose,30,44,N,Prognose,36,44,con,"Predicate(type='con', args=(Head(sentence=22, ...",Sein Alkoholkonsum verbessere seine Prognose a...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7d97d9b1d934840417d2935b8a240300ef4c026239ce562256103289c8b3a24a,retten,91,97,retten,die Servettiens,75,90,N,Servettiens,79,...,die Servettiens,75,90,N,Servettiens,79,90,pro,"Predicate(type='pro', args=(Head(sentence=23, ...",Zwei Minuten vor Schluss macht Goalie Descloux...
c5bdfdfa2dfd3aa81347456b8a032c69c3930173165813f53543c78eb46dd262,forderte,124,132,fordern,die Politik,88,99,N,Politik,92,...,neue Führungsstrukturen,100,123,N,Führungsstrukturen,105,123,pro,"Predicate(type='pro', args=(Head(sentence=0, t...",Beschleunigt wurde die Reform durch die jüngst...
f03e7b64ca4b13e8d4226284794eead23c0ae790d5c1d6a12717ec3e4161bf02,zugestimmt,133,143,zustimmen,Die Stimmberechtigten des Kantons Genf,0,38,N,Stimmberechtigten,4,...,einer Initiative der Gewerkschaften für einen ...,64,132,N,Initiative,70,80,pro,"Predicate(type='pro', args=(Head(sentence=0, t...",Die Stimmberechtigten des Kantons Genf hatten ...


In [47]:
dataframes[1].rel_type.value_counts()

con        13637
neutral    13596
pro        13395
Name: rel_type, dtype: int64

In [48]:
def transform_to_semeval_ssa_format(df):
    sent_list = []
    sentences = list(set(df.full_sentence_text.to_list()))
    for i, sentence in enumerate(sentences):
        opinions_list = []
        sub_df = df[df["full_sentence_text"] == sentence]
        for row in sub_df.itertuples():
            # skip neutral case, not needed here
            # if row.rel_type == "neutral":
            #    continue

            if FULL_PHRASE:
                # only-NPs
                opinions_list.append({
                    "Source": [[row.arg1], [f"{row.arg1_start}:{row.arg1_end}"]],
                    "Target": [[row.arg2_head], [f"{row.arg2_start}:{row.arg2_end}"]],
                    "Polar_expression": [[row.verb_form], [f"{row.verb_form_start}:{row.verb_form_end}"]],
                    "Polarity": "Positive" if row.rel_type == "pro" else "Negative" if row.rel_type == "con" else "Neutral",
                    # we currently do not care about intensity
                    "Intensity": "Average"
                })
            else:
                # non-NPs
                opinions_list.append({
                    "Source": [[row.arg1_head], [f"{row.arg1_head_start}:{row.arg1_head_end}"]],
                    "Target": [[row.arg2_head], [f"{row.arg2_head_start}:{row.arg2_head_end}"]],
                    "Polar_expression": [[row.verb_form], [f"{row.verb_form_start}:{row.verb_form_end}"]],
                    "Polarity": "Positive" if row.rel_type == "pro" else "Negative" if row.rel_type == "con" else "Neutral",
                    # we currently do not care about intensity
                    "Intensity": "Average"
                })

        if len(opinions_list) != 0:
            sent_list.append({
                "sent_id": str(i),
                "text": sentence,
                "opinions": opinions_list
            })
        else:
            continue
    return sent_list

In [36]:
# test the function
data = transform_to_semeval_ssa_format(dataframes[1])

In [ ]:
output_paths = [TRAIN_OUTPUT_PATH, TEST_OUTPUT_PATH, VAL_OUTPUT_PATH]

for op, df in zip(output_paths, dataframes):
    # call function for each... dev, test, train-json
    data = transform_to_semeval_ssa_format(df)
    # convert to JSON format.
    with open(op, mode="w") as f:
        json.dump(data, f, ensure_ascii=False)

In [41]:
# verify data
data[:1]

[{'sent_id': '0',
  'text': 'Und auch die unabhängigen Verwaltungsräte sowie Konzernleitung und Management stellten sich quer und rebellierten gegen die unerwünschte Übernahme .',
  'opinions': [{'Source': [['auch die unabhängigen Verwaltungsräte sowie Konzernleitung und Management'],
     ['4:77']],
    'Target': [['Übernahme'], ['120:146']],
    'Polar_expression': [['rebellierten'], ['101:113']],
    'Polarity': 'Negative',
    'Intensity': 'Average'}]}]